In [22]:
%load_ext autoreload
%autoreload 2

from BDDConvState import *
from enum import StrEnum
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
#from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

### TODO outstanding 

### 1. Continue the LLM analysis node 
### 2. Route the conversation based on the LLM analysis node in a router function which interprets the intention and the current state of the conversation
### 3. Have a TypedDict for the LLM analysis intention and the state
### 4. Lower case the user input by defapt and provide another variable for the raw user input

from BDDToLLM import GherkinStepType
from BDDConvState import LLMAnalysisIntention,LLMIntentionAnalysisResult




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of BDDConvState failed: Traceback (most recent call last):
  File "/home/ciprian/anaconda3/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/ciprian/anaconda3/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/ciprian/anaconda3/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/mnt/c/Users/cipri/OneDrive - University of Bucharest, Faculty of Mathematics and Computer Scien

In [9]:
# These are the conversation nodes that we will use to route the conversation
class ConvNode(StrEnum):
    #START = "START"
    #END = "END"
    WRITE_ASSISTANT_MSG ="write_assistant_msg" # This is the main node for writing messages from the assistant
    USER_INPUT = "user_input" # This is the main node for user input
    LLM_Check = "llm_check"
    #SCENARIO_DEF = "scenario_def"
    #SCENARIO_OUT_PATH = "scenarioOutPath"
    #SCENARIO_CODE_PATH = "scenarioCodePath"
    #SCENARIO_ADD_STEP = "scenarioAddStep"
    SCENARIO_ADD_EDIT = "scenarioAddEditStep"
    ADD_EDIT_BDD_STEP = "addEditBDDStep"
    BDD_TEST_MANAGEMENT = "bddTestManagement"
    
    #SUB_GRAPH_SCENARIO_EDIT = "sub_graph_scenario_edit"
    SUB_GRAPH_EDIT = "sub_graph_edit" # We use this for editing steps and scenarios
    SUB_GRAPH_ADD = "sub_graph_add" # We use this for adding steps
    SUB_GRAPH_SCENARIO_DEF = "sub_graph_scenario_def"
    SUB_GRAPH_SAVE_LOAD_DISCARD = "sub_graph_save_load_discard"
    

In [20]:
# Dummy tests to check the validation of the messages
analysis_intention = LLMIntentionAnalysisResult(intention=LLMAnalysisIntention.LLM_INTENTION_SAVE_SCENARIO)
# analysis_intention = LLMIntentionAnalysisResult(intention=LLMAnalysisIntention.LLM, BDD_STEP_TYPE=GherkinStepType.GIVEN,
#                                                 SCENARIO_EDIT_TYPE=ConvStepType.SCENARIO_DEF_PART3_CODE_PATH,
#                                                 raw_user_input="I want to save the scenario")



In [21]:
# All the function that are called on each node invocation defined here
# -----------------------------------------------------------
def write_assistant_msg(state: ConversationState):
    print("write_assistant_msg")
    
    curr_assist_msg = state.get_current_step_message()
    print(curr_assist_msg)
    
    return state

def user_input(state: ConversationState):
    print("user_input")
    
    # TODO: stream user input
    user_input = "placeholder user input"
    user_input = user_input.strip()
    return {'user_input_lowercased': user_input.lower(), 
            'user_input_casesensitive': user_input}
            

def llm_analysis_node (state: ConversationState):    
    return state.analyze_user_intention()     
    
    
    
def llm_check(state: ConversationState):
    
    next_node = None
    
    # Local function to get the next node based on the info from the current step if the current sub-graph could end or not
    def default_get_next_node(could_end: bool, _next_node: ConvNode = ConvNode.WRITE_ASSISTANT_MSG):
        if could_end:
            next_node = END
        else:
            next_node = _next_node
        
    if state.get_current_step_type() == ConvStepType.SCENARIO_DEF_PART1_SCENARIO_DEF:
        # TODO - need to use the tool function
        state.user_scenario = state.user_input
        could_end = state.check_transition_to_next_step(ConvStepType.SCENARIO_DEF_PART2_SCENARIO_OUTPUTPATH)
        next_node = default_get_next_node(could_end)
    
    elif state.get_current_step_type() == ConvStepType.SCENARIO_DEF_PART2_SCENARIO_OUTPUTPATH:
        # TODO - need to use the tool function
        state.test_case_path = state.user_input 
        could_end = state.check_transition_to_next_step(ConvStepType.SCENARIO_DEF_PART3_CODE_PATH)
        next_node = default_get_next_node(could_end)

    elif state.get_current_step_type() == ConvStepType.SCENARIO_DEF_PART3_CODE_PATH:
        could_end = state.check_transition_to_next_step(ConvStepType.GIVEN)        
        next_node = default_get_next_node(could_end)
        
    # This will reroute to the one of the steps above but keeping the rerouting previous step and state
    elif state.get_current_step_type() == ConvStepType.SCENARIO_DEF_EDIT_SELECTION:        
        # TODO: SHOULD ASK LLM but for now we just assume that the user wants to edit the scenario            
        if 'code' in state.user_input_lowercased:
            _ = state.check_transition_to_next_step(ConvStepType.SCENARIO_DEF_PART3_CODE_PATH)
            next_node = ConvNode.WRITE_ASSISTANT_MSG
            
        elif any(['path', 'output', 'out', 'save'] in state.user_input_lowercased):
            _ = state.check_transition_to_next_step(ConvStepType.SCENARIO_DEF_PART2_SCENARIO_OUTPUTPATH)
            next_node = ConvNode.WRITE_ASSISTANT_MSG
        elif any(['scenario', 'description'] in state.user_input_lowercased):
            _ = state.check_transition_to_next_step(ConvStepType.SCENARIO_DEF_PART1_SCENARIO_DEF)
            next_node = ConvNode.WRITE_ASSISTANT_MSG
        else:
            _ = state.check_transition_to_next_step(ConvStepType.ASSISTANT_DOES_NOT_UNDERSTAND)
            next_node = ConvNode.WRITE_ASSISTANT_MSG    
         
    assert next_node is not None, f"llm_check: next_node is None, current_step_type: {state.get_current_step_type()}"
    return state

# def scenarioDefStep(state: ConversationState):
#     print("scenario_def")
#     return state

def scenarioAddEditStep(state: ConversationState):
    print("scenarioAddStep")
    return state

# def scenarioEditStep(state: ConversationState):
#     print("scenarioEditStep")
#     return state

def addEditBDDStep(state: ConversationState):
    print("addEditBDDStep")
    return state

# Discard, load, save
def bdd_test_management(state: ConversationState):
    print("bdd_test_management")
    return state

# -----------------------------------------------------------


In [28]:

memory = MemorySaver()
builder = StateGraph(ConversationState, memory)

# THis is the graph handling the scenario editing, adding the source code, references, paths to the output etc.
# -----------------------------------------------------------
sub_graph_scenario_builder = StateGraph(ConversationState, memory)
sub_graph_scenario_builder.add_node(ConvNode.USER_INPUT, user_input)
#sub_graph_scenario_builder.add_node(ConvNode.SCENARIO_EDIT, scenarioEditStep)
sub_graph_scenario_builder.add_node(ConvNode.SCENARIO_ADD_EDIT, scenarioAddEditStep)
#sub_graph_builder.add_node(ConvNode.SCENARIO_ADD_STEP)
sub_graph_scenario_builder.add_node(ConvNode.LLM_Check, llm_check)
sub_graph_scenario_builder.add_node(ConvNode.WRITE_ASSISTANT_MSG, write_assistant_msg)

sub_graph_scenario_builder.add_edge(START, ConvNode.WRITE_ASSISTANT_MSG)
sub_graph_scenario_builder.add_edge(ConvNode.WRITE_ASSISTANT_MSG, ConvNode.USER_INPUT)
sub_graph_scenario_builder.add_edge(ConvNode.USER_INPUT, ConvNode.LLM_Check)
sub_graph_scenario_builder.add_conditional_edges(ConvNode.LLM_Check, {ConvNode.WRITE_ASSISTANT_MSG, END})

# Now compile graph
sub_graph_scenario = sub_graph_scenario_builder.compile() 
display(Image(sub_graph_scenario.get_graph().draw_mermaid_png()))
    

ValueError: Node `user_input` is not reachable

In [ ]:

# This is the graph handling the adding of steps
sub_graph_bddsteps_builder = StateGraph(ConversationState, memory)
sub_graph_bddsteps_builder.add_node(ConvNode.ADD_EDIT_BDD_STEP, addEditBDDStep)
sub_graph_bddsteps_builder.add_node(ConvNode.WRITE_ASSISTANT_MSG, write_assistant_msg)
sub_graph_bddsteps_builder.add_node(ConvNode.LLM_Check)
sub_graph_scenario_builder.add_node(ConvNode.ADD_EDIT_BDD_STEP, addEditBDDStep)

# Now compile and render
sub_graph_bddsteps = sub_graph_bddsteps_builder.compile()
display(Image(sub_graph_bddsteps.get_graph().draw_mermaid_png()))

In [ ]:
main_graph_builder = StateGraph(ConversationState, memory)
#main_graph.add_node(ConvNode.START) # Appears that both are added by default
#main_graph.add_node(ConvNode.END)
main_graph_builder.add_node(ConvNode.USER_INPUT)
main_graph_builder.add_node(ConvNode.SUB_GRAPH_EDIT, sub_graph_scenario_builder.

In [ ]:
def write_step(state: ConversationState):
    print(f"The current state is: {state.get_current_step_type()}")
    print(state.get_current_step_message()) # this is the message that the user sees
    return 
    

In [5]:
builder.add_node("WriteStep", write_step)
builder.add_node("user_input", user_input)
builder.add_node("llm_check_input", llm_check_input)
builder.add_node("scenarioDef", scenarioDef)
builder.add_node("scenarioOutPath", scenarioOutPath)
builder.add_node("scenarioCodePath", scenarioCodePath)
builder.add_node("add_step", add_step)
builder.add_node("show_test", show_test)
builder.add_node("save_test", save_test)
builder.add_node("discard_test", discard_test)
builder.add_node("cleanup", cleanup)
builder.add_node("call_tool", call_tool)


NameError: name 'write_step' is not defined